In [1]:
!pip install tensorflow

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Add, GlobalAveragePooling2D,\
	Dense, Flatten, Conv2D, Lambda,	Input, BatchNormalization, Activation
from tensorflow.keras.optimizers import schedules, SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

2024-02-10 18:20:25.221403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 18:20:25.221523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 18:20:25.378418: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_path = r'/kaggle/input/split-preprocessed-dataset/dataset1/train'
test_path = r'/kaggle/input/split-preprocessed-dataset/dataset1/test'

In [4]:
wounds = os.listdir(train_path)
print(wounds)

['Abrasions', 'Burn_or_Stabs', 'Cut_or_Laceration', 'Bruises']


In [5]:
nums = {}
for wound in wounds:
    nums[wound] = len(os.listdir(train_path + '/' + wound))
    
print(nums)

{'Abrasions': 68, 'Burn_or_Stabs': 65, 'Cut_or_Laceration': 88, 'Bruises': 97}


In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.4,
    zoom_range = 0.4,
    horizontal_flip = True,
    fill_mode = 'nearest',
    vertical_flip = True,
    width_shift_range = 0.2,
    rotation_range = 20
)
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size = (64, 64),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'categorical'
)

Found 318 images belonging to 4 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(
    test_path,
    target_size = (64, 64),
    color_mode = 'grayscale',
    batch_size = 32,
    class_mode = 'categorical'
)

Found 82 images belonging to 4 classes.


In [8]:
#Initialising the neural network
cnn = tf.keras.models.Sequential()

In [9]:
#First Convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64, 64, 1]))

In [10]:
#A dropout layer to avoid overfitting
cnn.add(tf.keras.layers.Dropout(.4, input_shape=(2,), seed = 0))

In [11]:
#A max-pooling layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [12]:
#Adding a secdon convolutional layer with max-pooling layer
cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

In [13]:
#A flattening layer
cnn.add(tf.keras.layers.Flatten())

In [14]:
#Fully connected layer with 64 neurons (Dense layer)
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

In [15]:
#Output layer with 36 neurons as there are 36 different digits in ISL
cnn.add(tf.keras.layers.Dense(units = 4, activation = 'softmax'))

In [16]:
#A summary chart for all the neurons, their number of filters and number of units
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 dropout (Dropout)           (None, 62, 62, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 12544)             0

In [17]:
#Compilation of the CNN
cnn.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [18]:
#Model training
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 75)

Epoch 1/75


2024-02-10 18:20:39.534263: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1707589242.774689     124 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 [==============================] - 8s 279ms/step - loss: 1.5372 - accuracy: 0.2327 - val_loss: 1.3756 - val_accuracy: 0.2805
Epoch 2/75
10/10 [==============================] - 1s 111ms/step - loss: 1.3839 - accuracy: 0.3050 - val_loss: 1.3784 - val_accuracy: 0.3049
Epoch 3/75
10/10 [==============================] - 1s 110ms/step - loss: 1.3777 - accuracy: 0.3050 - val_loss: 1.3750 - val_accuracy: 0.3049
Epoch 4/75
10/10 [==============================] - 1s 110ms/step - loss: 1.3646 - accuracy: 0.3019 - val_loss: 1.3647 - val_accuracy: 0.3171
Epoch 5/75
10/10 [==============================] - 1s 112ms/step - loss: 1.3596 - accuracy: 0.3050 - val_loss: 1.3634 - val_accuracy: 0.3171
Epoch 6/75
10/10 [==============================] - 1s 148ms/step - loss: 1.3549 - accuracy: 0.3396 - val_loss: 1.3334 - val_accuracy: 0.3780
Epoch 7/75
10/10 [==============================] - 1s 116ms/step - loss: 1.3402 - accuracy: 0.3868 - val_loss: 1.3222 - val_accuracy: 0.4390
Epoch 8/75
10/10 

In [20]:
import cv2

In [74]:
# train_path = r'/kaggle/input/split-preprocessed-dataset/dataset1/train'
tester_path = r'/kaggle/input/image2/abrasion2.jpg'

In [75]:
test_img = cv2.imread(tester_path)
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
test_img = cv2.resize(test_img, (64, 64))

In [76]:
test_image = cv2.imread(tester_path)
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
test_image = cv2.resize(test_image, (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
p = training_set.class_indices
print(p)
test_image = np.array(test_image)
test_image = test_image/255.0
result = cnn.predict([test_image])

{'Abrasions': 0, 'Bruises': 1, 'Burn_or_Stabs': 2, 'Cut_or_Laceration': 3}
1/1 [==============================] - 0s 21ms/step


In [77]:
result

array([[0.31594965, 0.24133797, 0.0953256 , 0.34738678]], dtype=float32)

In [78]:
m = result.max()
print(m)
for i in range(len(result)):
    if result[0][i] == m:
        val = result[i]
    
print(val)

0.34738678
0


In [79]:
cnn.save('cnn.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
